# CS533 Assignment 2
---
### Course No: CS533
### Assingment No: 2

<br>

### Student Name: Hasan Alp Caferoğlu
### Student ID: 22203991
### Email Address:  alp.caferoglu@bilkent.edu.tr


---



In [2]:
# Install necessary packages
!pip install rank-bm25
!pip install ipython-autotime
%load_ext autotime

time: 344 µs (started: 2023-11-26 17:00:32 +00:00)


In [3]:
from google.colab import drive
drive.mount('/content/drive')
CISI_documents_path = '/content/drive/MyDrive/1.MS/CS533_IRS/assignments/assignment2/implementation/CISI/documents.csv'
CISI_queries_path = '/content/drive/MyDrive/1.MS/CS533_IRS/assignments/assignment2/implementation/CISI/queries.csv'
CISI_ground_truth_path = '/content/drive/MyDrive/1.MS/CS533_IRS/assignments/assignment2/implementation/CISI/ground_truth.csv'

Mounted at /content/drive
time: 19.8 s (started: 2023-11-26 17:00:32 +00:00)


In [4]:
# Imports
import pandas as pd
from gensim.models import Word2Vec
import gensim.downloader as api
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from rank_bm25 import BM25Okapi
import nltk
from nltk.corpus import stopwords
import string

# Downloading NLTK resources
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

time: 3.91 s (started: 2023-11-26 17:00:52 +00:00)


In [5]:
# Reading CSV files into Pandas DataFrames
documents_df = pd.read_csv(CISI_documents_path)
queries_df = pd.read_csv(CISI_queries_path)
ground_truth_df = pd.read_csv(CISI_ground_truth_path)

time: 1.51 s (started: 2023-11-26 17:00:56 +00:00)


In [6]:
# Tokenization function
def tokenizerFunction(textData):
  """
    Splitting of a sentence into its words and returns list of words

    Parameters:
    - textData: list, sentence that will be tokenized

    Returns:
    - tokens, list of tokens
  """
  tokens = textData.split()
  return tokens

time: 678 µs (started: 2023-11-26 17:00:58 +00:00)


In [7]:
# Preprocessing function
def preprocessFunction(textData):
  """
    Applying preprocessing to the given sentence.
    Preprocessing involves removing stop words, removing punctuation marks
    and lowercasing the text.

    Parameters:
    - textData: list, sentence that will be preprocessed

    Returns:
    - processedTextData as string
  """
  # Converting text to lowercase
  if isinstance(textData, str):
    textData = textData.lower()

    # Removing punctuations
    # textData = textData.translate(str.maketrans('', '', string.punctuation)) # converts "don't" to "dont"
    textData = textData.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation))) # converts "don't" to "don t"

    # Removing stopwords
    stopWordsSet = set(stopwords.words('english'))
    tokens = tokenizerFunction(textData)
    tokens = [token for token in tokens if token not in stopWordsSet]

    processedTextData = ' '.join(tokens)
    return processedTextData
  else:
    # if the text data is NaN return empty string
    return ''

time: 5.63 ms (started: 2023-11-26 17:00:58 +00:00)


In [8]:
# Preprocessing 'text' column in each DataFrame
documents_df['processedText'] = documents_df['text'].apply(preprocessFunction)
documents_df['processedTitle'] = documents_df['title'].apply(preprocessFunction)
queries_df['processedText'] = queries_df['text'].apply(preprocessFunction)

# Storing tokens of processed text
documents_df['processedTextTokens'] = documents_df['processedText'].apply(tokenizerFunction)
documents_df['processedTitleTokens'] = documents_df['processedTitle'].apply(tokenizerFunction)
queries_df['processedTextTokens'] = queries_df['processedText'].apply(tokenizerFunction)

time: 1.27 s (started: 2023-11-26 17:00:58 +00:00)


In [9]:
# Combining processed document titles, document texts and query texts for train set
trainDataSetTokenized =  documents_df['processedTitleTokens'].tolist() + documents_df['processedTextTokens'].tolist() + queries_df['processedTextTokens'].tolist()
trainDataSet = documents_df['processedTitle'].tolist() + documents_df['processedText'].tolist() + queries_df['processedText'].tolist()

time: 2.17 ms (started: 2023-11-26 17:00:59 +00:00)


In [10]:
# Initializing Gensim Word2Vec model
model = Word2Vec(
    window=10, # 10 words before the target word and 10 words after it
    min_count=2, # at least 2 words must be present in the sentence
    workers=2, # number of used cpu threads for training,
    vector_size=300, # vector size is set to 300 to equalize with the google model
    epochs=50
)

# Building vocabulariy (building list of unique words)
model.build_vocab(trainDataSetTokenized)
model.train(trainDataSetTokenized,
            total_examples=model.corpus_count,
            epochs=model.epochs
)

(5058147, 5495550)

time: 46 s (started: 2023-11-26 17:00:59 +00:00)


In [11]:
# Downlading pre-trained model "word2vec-google-news-300"
preTrainedGoogleNewsModel = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded
time: 8min 59s (started: 2023-11-26 17:01:45 +00:00)


In [12]:
def embeddingComputationFunction(nlTokens, wvModel):
  """
    Deriving the embedding for an input (a query or a document) by averaging
    the embeddings of its constituent words.

    Parameters:
    - nlTokens: list, the tokens of input query or document for which to compute the embedding.
    - wvModel: KeyedVectors, model that is used for computing embeddings

    Returns:
    - np.ndarray, embedding vector for a given input.
  """

  # Filtering words that are not present in the model
  nlTokens = [token for token in nlTokens if token in wvModel]

  # Returning zero vector if there is no words that present in the model
  if not nlTokens:
    return np.zeros(wvModel.vector_size)

  tokensEmbeddingList = [wvModel[token] for token in nlTokens]
  # Note that average embedding is found by averaging along the column
  avgEmbedding = np.mean(tokensEmbeddingList, axis=0)

  return avgEmbedding

time: 1.09 ms (started: 2023-11-26 17:10:45 +00:00)


In [13]:
# instantiate the TF-IDF vectorizer object
tfidfVectorizer = TfidfVectorizer()
# Learning vocabulary and idf
tfidfVectorizer.fit(trainDataSet)
tfidfTrainSetVector = tfidfVectorizer.transform(trainDataSet) # tfidfVector is scipy sparse matrix

time: 672 ms (started: 2023-11-26 17:10:45 +00:00)


In [14]:
# BM25 Initializing
bm25 = BM25Okapi(documents_df['processedTextTokens'])

time: 119 ms (started: 2023-11-26 17:10:45 +00:00)


In [15]:
# Calculating TF-IDF score for query-document pair
def tfidfScoreQueryDocPair(query, doc):
  """
    Obtaining tfidf score for a document query pair

    Parameters:
    - query: string, considered query
    - docId: string, considered doc

    Returns:
    - float, tfidf score for query-document pair
  """
  tfidfScore = np.dot(tfidfVectorizer.transform([doc]).toarray(), tfidfVectorizer.transform([query]).toarray().T).sum()
  return tfidfScore

# Example for TF-IDF score
tfidfScoreQueryDocPair(queries_df['processedText'][0], documents_df['processedText'][758])

0.15873646896054894

time: 21.4 ms (started: 2023-11-26 17:10:45 +00:00)


In [16]:
def bm25ScoreQueryDocPair(query, docId):
  """
    Obtaining BM25 score for a document query pair

    Parameters:
    - query: string, the query which related documents wil be retrieved
    - docId: int, document ID that is compared with given query

    Returns:
    - float, BM25 score for query-document pair
  """
  scoreList = bm25.get_scores(query.split())
  # normalization of score is done by dividing obtained score with max score
  bm25Score = scoreList[docId-1] / max(scoreList)
  return bm25Score

# Example for BM25 Score
bm25score = bm25ScoreQueryDocPair(queries_df['processedText'][0], 759)
print(bm25score)


0.8012794563366724
time: 42.3 ms (started: 2023-11-26 17:10:45 +00:00)


In [17]:
def embeddingScoreQueryDocPair(query, doc, model):
  """
    Obtaining embedding score for a document query pair

    Parameters:
    - query: string, considered query
    - doc: string, considered doc
    - model: word2vec model that is wanted to be used

    Returns:
    - float, embedding score for query-document pair
  """
  queryEmbeddings = embeddingComputationFunction(query.split(), model)
  docEmbeddings = embeddingComputationFunction(doc.split(), model)
  # applying cosine similarity
  if (np.linalg.norm(queryEmbeddings) * np.linalg.norm(docEmbeddings)) != 0:
    embeddingScore = np.dot(queryEmbeddings, docEmbeddings) / (np.linalg.norm(queryEmbeddings) * np.linalg.norm(docEmbeddings))
    return embeddingScore
  else:
    return 0

# Example for embedding score
embeddingScore = embeddingScoreQueryDocPair(queries_df['processedText'][0], documents_df['processedText'][758], model.wv)
print(embeddingScore)
embeddingScore = embeddingScoreQueryDocPair(queries_df['processedText'][0], documents_df['processedText'][758], preTrainedGoogleNewsModel)
print(embeddingScore)

0.6386269
0.68639153
time: 14.6 ms (started: 2023-11-26 17:10:45 +00:00)


In [18]:
def combinedScoreQueryDocPair(doc, query, docId, embeddingModel, tfidfWeight, bm25Weight, embeddingWeight):
  """
    Obtaining total combined score for a document query pair

    Parameters:
    - doc: string, context of the doc
    - query: string, query text
    - docId: int, ID of considered doc
    - embeddingModel: word2vec model that is wanted to be used
    - tfidfWeight: float, weight for tfidf score
    - bm25Weight: float, weight for bm25 score
    - embeddingWeight: float, weight for embedding score

    Returns:
    - float, combined score for query-document pair
  """

  tfidfScore = tfidfScoreQueryDocPair(query, doc)
  bm25Score = bm25ScoreQueryDocPair(query, docId)
  embeddingScore = embeddingScoreQueryDocPair(query, doc, embeddingModel)
  combinedScore = tfidfWeight * tfidfScore + bm25Weight * bm25Score + embeddingWeight * embeddingScore
  return combinedScore

# Example for combined score
docId = 759
queryId = 1
doc = documents_df['processedText'][docId-1]
query = queries_df['processedText'][queryId-1]
combinedScore = combinedScoreQueryDocPair(doc, query, docId, model.wv, 0.33, 0.33, 0.33)
print(combinedScore)
combinedScore = combinedScoreQueryDocPair(doc, query, docId, preTrainedGoogleNewsModel, 0.33, 0.33, 0.33)
print(combinedScore)

0.5275521236008373
0.5433144610470501
time: 124 ms (started: 2023-11-26 17:10:45 +00:00)


In [19]:
def retrievalSystem(docDataFrame, queryDataFrame, queryId, embeddingModel, tfidfWeight, bm25Weight, embeddingWeight):
  docScores = []
  query = queryDataFrame['processedText'][queryId-1]
  for docIndex in range(len(docDataFrame)):
    docId = docIndex+1
    doc = docDataFrame['processedText'][docId-1]
    combinedScore = combinedScoreQueryDocPair(doc, query, docId, embeddingModel, tfidfWeight, bm25Weight, embeddingWeight)
    docScores.append((docId, combinedScore))

  sortedDocScores = sorted(docScores, key=lambda x: x[1], reverse=True)
  top10 = [item[0] for item in sortedDocScores[:10]]
  return top10

# Examine the retrievalSystem
# top10ForQ1 = retrievalSystem(documents_df, queries_df, 1, model.wv, 0.33, 0.33, 0.33 )
# print(top10ForQ1)
# top10ForQ1 = retrievalSystem(documents_df, queries_df, 1, preTrainedGoogleNewsModel, 0.33, 0.33, 0.33 )
# print(top10ForQ1)


time: 1.23 ms (started: 2023-11-26 17:10:46 +00:00)


In [20]:
def calculateMAPForSystem(groundTruthDataFrame, docDataFrame, queryDataFrame, embeddingModel, tfidfWeight, bm25Weight, embeddingWeight):
  # applying MAP at 10
  avgPrecisionForQueries = []
  # Obtaining queries and count of relevant documents
  queryAndRelDocNum =  groundTruthDataFrame['query_id'].value_counts().reset_index().rename(columns={'query_id': 'rel_doc_count', 'index': 'query_id'})
  queryAndRelDocNum=queryAndRelDocNum.sort_values(by='query_id').reset_index(drop=True)
  print(queryAndRelDocNum) # DELETE LATER
  for qIndex in range(len(queryAndRelDocNum)):
    queryId = queryAndRelDocNum.loc[qIndex, 'query_id']
    relevantDocIds = (groundTruthDataFrame[groundTruthDataFrame['query_id'] == queryId])['doc_id'].tolist()
    top10RetrievedDocIds = retrievalSystem(docDataFrame, queryDataFrame, queryId, embeddingModel, tfidfWeight, bm25Weight, embeddingWeight)
    # print('top 10 retrieved docs for query id ', queryId, ": ") # DELETE LATER
    # print(top10RetrievedDocIds) # DELETE LATER

    relevanceCount = 0
    precisionSum = 0
    for i in range(len(top10RetrievedDocIds)):
      retrievedDocId = top10RetrievedDocIds[i]
      if retrievedDocId in relevantDocIds:
        relevanceCount+=1
        # print(relevanceCount, "/", i+1, " = ", relevanceCount / (i+1) ) # DELETE LATER
        precisionSum += relevanceCount / (i+1)

    # print('precision sum: ', precisionSum) # DELETE LATER
    avgPrecision = precisionSum / len(relevantDocIds)
    print('Average Precision for query id ', queryId, ": ", avgPrecision) # DELETE LATER
    avgPrecisionForQueries.append(avgPrecision)
    print("-"*30) # DELETE LATER

  print(" Sum of average precisions: ", sum(avgPrecisionForQueries)) # DELETE LATER
  print("Number of queries considered: ", len(avgPrecisionForQueries)) # DELETE LATER
  MAP = sum(avgPrecisionForQueries) / len(avgPrecisionForQueries)
  print('MAP for the system: ', MAP)
  print("-"*30)
  return MAP

time: 2.11 ms (started: 2023-11-26 17:10:46 +00:00)


In [21]:
embeddingModels = [model.wv, preTrainedGoogleNewsModel]
systemsWeightList = [(0.4, 0.25, 0.35)]

systemsDict = {}
sysCount = 1
for index in range(len(systemsWeightList)):
  system = {'embeddingModel': 0, 'weights': systemsWeightList[index]}
  systemsDict[f'{sysCount}'] = system
  sysCount+=1

  # if embeddingWeight is larger than 0, there must 2 systems with both our trained model and pretrained model
  if systemsWeightList[index][2] > 0:
    systemWithAnotherModel = {'embeddingModel': 1, 'weights': systemsWeightList[index]}
    systemsDict[f'{sysCount}'] = systemWithAnotherModel
    sysCount+=1


# print(systemsDict)
# print(len(systemsDict))
for systemId in range(1, len(systemsDict)+1):
  system = systemsDict[f'{systemId}']
  weights = system['weights']
  tfidfWeight, bm25Weight, embeddingWeight = system['weights']
  embeddingModelNo = system['embeddingModel']
  MAP = calculateMAPForSystem(ground_truth_df, documents_df, queries_df, embeddingModels[embeddingModelNo], tfidfWeight, bm25Weight, embeddingWeight)
  system['MAP'] = MAP
  print(f'System {systemId}: \n MAP: {MAP} Weights: {weights} Embedding Model: {embeddingModels[embeddingModelNo]} \n')


# Printing System Performances into txt file
systemPerformanceFilePath = '/content/drive/MyDrive/1.MS/CS533_IRS/assignments/assignment2/implementation/systemPerformances6.txt'
with open(systemPerformanceFilePath, 'w') as fileHandle:
  for systemId, system in systemsDict.items():
    weights = system['weights']
    MAP = system['MAP']
    embeddingModelId = system['embeddingModel']
    if embeddingModelId == 1:
      embeddingModel = 'Trained Model with CISI Dataset'
    else:
      embeddingModel = 'word2vec-google-news-300'
    line = f'System {systemId}: \n MAP: {MAP} Weights: {weights} Embedding Model: {embeddingModel} \n'
    fileHandle.write(line)


    query_id  rel_doc_count
0          1             46
1          2             26
2          3             44
3          4              8
4          5             24
..       ...            ...
71       101              1
72       102             24
73       104             11
74       109             71
75       111              6

[76 rows x 2 columns]
Average Precision for query id  1 :  0.10429606625258799
------------------------------
Average Precision for query id  2 :  0.0
------------------------------
Average Precision for query id  3 :  0.11875
------------------------------
Average Precision for query id  4 :  0.08660714285714285
------------------------------
Average Precision for query id  5 :  0.013888888888888888
------------------------------
Average Precision for query id  6 :  0.0
------------------------------
Average Precision for query id  7 :  0.0
------------------------------
Average Precision for query id  8 :  0.005555555555555556
--------------------------

time: 2h 43min 27s (started: 2023-11-26 17:10:46 +00:00)
